<a href="https://colab.research.google.com/github/johngrahamreynolds/QuantumComputing/blob/main/canonical_algorithms/deutsch%E2%80%93jozsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The Deutsch and Deutsch–Jozsa algorithms
#### This Colab implements the Deutsch algorithm using Cirq before discussing its generalization, the Deutsch–Jozsa algorithm. Lastly, we examine the ability of Anthropic's LLM 'Claude' to generate efficient and correct code for the implementation of this algorithm.

###### johngrahamreynolds@gmail.com

First we install Cirq and import relevant packages to our workspace

In [84]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq

import random
from typing import List, Iterable

## Introduction and motivation for these algorithms

Given a black box function $f: \{0,1\}^n → \{0,1\}$, promised to be balanced or constant,
we can show that quantum algorithms can determine its global nature (that is, constant or balanced) much faster than classical algotithms.
A constant function returns either $0$ or $1$ for all inputs while a balanced function returns half $1$s, half $0$s.
We call the function an oracle, and send it information to compute the value of the function for an input.
We refer to this as querying the oracle.

In the case of the Deutsch and the Deutsch-Jozsa algorithms, we can determine the global
property of the function with just a single query to the oracle by desinging a
quantum circuit that utilizes quantum parallelism and a phase kickback trick to encode the global property.
The phase kickback trick utilizes quantum intereference, either constructive or desctructive, when we measure.

A classical computer does not have the same computational ability. The minimum number of queries to the oracle to discern the function's global property on a classical computer is garuanteed to be more than the number of queries conducted by quantum algorithms.
In the case of the $f: \{ 0,1\} → \{0,1\}$, a classical algorithm needs a minmum 2 queries to the oracle. In the case of $f: \{ 0,1\}^n → \{0,1\}$, a classical algorithm needs a minimum of $2^{n-1} + 1$ queries to the oracle.
In both cases, the quantum Deutsch and Deutsch-Jozsa algorithms, respectively, need only a single query.
This demonstrates in the generalized case that the quantum Deutsch-Jozsa algorithm provides exponential speedup over any classical computer's ability to solve this problem.

## The Deutsch algorithm

Simplification to the case of $n = 1$ gives us four possible functions represented by the truth table below

| $x$ | $f_{0}$($x$) | $f_{1}$($x$) | $f_{x}$($x$) | $f_{\bar{x}}$($x$) |
|---|----------------|--------------|--------------|--------------------|
| 0 |    0           |  1           |   0          |  1                 |
| 1 |     0          |    1         |   1          |   0                |

The global nature of this function, whichever is chosen, can be found with just a single query to the oracle as discussed above.

### Problem Statement

Bob and Alice wish to see how fast they can solve a problem together while Bob hides information from Alice. Bob tells Alice that he is going to construct a function of the type described above. That is, he constructs a function of the form $f: \{ 0,1\}^n → \{0,1\}$ and promises Alice that it will be either constant or balanced. Bob asks Alice to discern the global propety of the function in as few queries as possible. She can query his black box function by sending it $n$ bits of information each time. The pair agree to the case of $n=1$ to begin.

Alice has read "Mike and Ike's" book Quantum Computation and Quantum Information and decides that she can build a quantum circuit to implement Deutsch's algorithm to solve this problem.


## Implementation of the Deutsch algorithm

In [85]:
# First, Bob builds a custom gate to encode the operation of his blackbox function to be used in the Alice's circuit
# The blackbox oracle, unbeknowst to the Alice, knows whether the function is constant or balanced based on Bob's choice
# Given the case of n=1, there are 4 functions that can exist as exhibited in the truth table above
# Bob thus encodes his blackbox to operate in one of 4 ways on the qubit that Alice sends into his function. His choice of function determines which computation is run
# Note that while Bob is aware of the function's global property, he only computes the function's values for the n (qu)bits Alice sends him. He does not tell her the answer explicitly

class bobs_deutsch_oracle(cirq.Gate):

  def __init__(self, function_type: str):
        super(bobs_deutsch_oracle, self)
        self.function_type = function_type

  def _num_qubits_(self) -> int:
    return 2

  def _decompose_(self, qubits):
    if self.function_type == "constant_0":
      yield cirq.IdentityGate(2).on(qubits[0], qubits[1])
    elif self.function_type == "constant_1":
      yield cirq.X(qubits[1])
    elif self.function_type == "balanced_x":
      yield cirq.CNOT(qubits[0], qubits[1])
    elif self.function_type == "balanced_notx":
      yield [cirq.CNOT(qubits[0], qubits[1]), cirq.X(qubits[1])]

  def _circuit_diagram_info_(self, args):
        return ["deutsch"] * self._num_qubits_()


In [86]:
# Now Alice designs her Deutsch quantum circuit
# She builds it to interact with the black box oracle that Bob will configure based on his choice of the function
# Wisely, she designs the circuit in the form of Deutsch's alrogithm to employ quantum parallelism, a phase kickback trick, and finally quantum interference
# By designing her algorithm this way, she need query Bob's blackbox function within her circuit only a single time to discern the function's global nature

def alices_deutsch_algorithm(function_type: str, repetitions: int = 1) -> int:

  query_register = cirq.LineQubit(0)
  answer_qubit = cirq.LineQubit(1)

  # flip the answer qubit to |1>
  moment_0 = cirq.Moment(cirq.X(answer_qubit))

  # apply Hadamard gate to the qubit in the query register and the answer qubit
  moment_1 = cirq.Moment(cirq.H.on_each([query_register, answer_qubit]))

  # apply the quantum deutsch oracle
  moment_2 = [cirq.Moment(bobs_deutsch_oracle(function_type).on(query_register, answer_qubit))]

  # apply Hadamard again to the qubit in the query register
  moment_3 = cirq.Moment(cirq.H.on_each(query_register))

  # measure the query registrer
  moment_4 = cirq.Moment(cirq.measure(query_register, key='result'))

  circuit = cirq.Circuit((moment_0, moment_1, moment_2, moment_3, moment_4))

  print(circuit)
  sim = cirq.Simulator()
  return sim.run(circuit, repetitions=repetitions)


In [87]:
# create the list of possible black box functions for the oracle
possible_deutsch_functions = ["constant_0", "constant_1", "balanced_x", "balanced_notx"]

# Alice's dictionary that intelligently decodes her quantum algorithm's results
decoding = {0: "constant", 1: "balanced"}

# Bob picks one of the possible functions at random
bobs_choice = random.choice(possible_deutsch_functions)
print("Bob has randomly chosen the " + bobs_choice + " function.\n")

# Alice runs her algorithm with a single query to the oracle to discern the global nature of the function
print("Making intelligent use of Bob's black box oracle, Alice constructs her quantum ciruit as so: \n")
response = alices_deutsch_algorithm(bobs_choice)
print(f"\nShe runs her quantum algorithm and records a final qubit query register |(f(0) + f(1) mod 2)> in the state: |{response.measurements['result'][0][0]}>.\n")
print(f"She determines immediately, with just one query to the oracle, that the function is {decoding[response.measurements['result'][0][0]].upper()}.\n")

Bob has randomly chosen the constant_0 function.

Making intelligent use of Bob's black box oracle, Alice constructs her quantum ciruit as so: 

0: ───────H───deutsch───H───M('result')───
              │
1: ───X───H───deutsch─────────────────────

She runs her quantum algorithm and records a final qubit query register |(f(0) + f(1) mod 2)> in the state: |0>.

She determines immediately, with just one query to the oracle, that the function is CONSTANT.



In [89]:
# Indeed, one might wonder whether Alice just got lucky? Did she just have a high probabilty of being correct?
# The answer is no. She has engineered her algorithm to be fully deterministic

# We can illustrate this as so:
# Allow Bob to again randomly choose a function, and, this time, allow Alice to run her algorithm #(repetitions) times
# We find that Alice is correct in all #(repetitions) attempts. Exponentially increasing the number of repeititions will never change any single output of the algorithm

repetitions = 15
# repetitions = 500
bobs_choice = random.choice(possible_deutsch_functions)
print("This time, Bob has randomly chosen the " + bobs_choice + " function.\n")

print("Again, Alice's quantum ciruit is: \n")

response_2 = alices_deutsch_algorithm(bobs_choice, repetitions=repetitions)
print(f"\nAlice runs her algorithm {repetitions} times and determines in each case that that the function is:")
for i in range(repetitions):
  print(decoding[response_2.measurements['result'][0][0]].upper())

This time, Bob has randomly chosen the balanced_notx function.

Again, Alice's quantum ciruit is: 

0: ───────H───deutsch───H───M('result')───
              │
1: ───X───H───deutsch─────────────────────

Alice runs her algorithm 15 times and determines in each case that that the function is:
BALANCED
BALANCED
BALANCED
BALANCED
BALANCED
BALANCED
BALANCED
BALANCED
BALANCED
BALANCED
BALANCED
BALANCED
BALANCED
BALANCED
BALANCED


## The Deutsch-Jozsa algorithm

For the general case of any integer $n > 1$, the Deutsch-Jozsa algorithm can still solve this problem with only a single query to the oracle. We expand our query register to $n$ qubits, but otherwise the gates in our quantum circuit are identical with the generalization that the Deutsch gate (now called the Deutsch-Jozsa gate) acts on the entire $n$ qubit query register. Rather than considering all possible constant or balanced mappings from the domain $\{0,1\}^{n}$ to the target space $\{0,1\}$ (as we did in the case of $n=1$), we can instead simply have Bob secretly select from only a small subset of the possible functions. Trying to configure the blackbox to operate exactly on each $n$-bitstring input as we did before turns into a unwiedly task very, very quickly. We can examine this as follows.

In the generalized case, there still exist only two constant functions: $f_{0}: \{ 0,1\}^n → \{0\}$ and $f_{1}: \{ 0,1\}^n → \{1\}$. The case of balanced functions is much trickier. Given $2^n$ possible bit strings, a balanced function must map exactly $2^{n-1}$ of these to $0$ and the other $2^{n-1}$ to $1$. That is, we have $2^{n}$ choose $2^{n-1}$ different ways of constructing a balanced function. We can define this mathematically with a function $f_b(n)$ that counts the number of possible balanced functions for any $n$ by expanding the binomial coefficient as  

$ f_b(n) = { 2^{n} \choose 2^{n-1}} = \frac{2^{n}!}{2^{n-1}!(2^{n}-2^{n-1})!}$

This function grows both exponentially and factorially. In the example of the simplified Deutsch algorithm from before we find the expected result of $f_b(1)=2$ as we saw in the truth table. For the next non-trivial cases beyond that, we have first a modest selection of $f_b(2) = 6$ and then a far less friendly $f_b(3) = 70$ possibilities for distinct balanced functions. Already for the case $n=9$ we have that $f_b(9) ≈ 10^{152}$, which is indeed approximately $10^{70}$ times larger than the current hypothetical upperbound on the number of atoms in the universe at ~ $10^{82}$ atoms. Forgetting for a brief second the restrictions of quantum mechanics and its associated trickery, even if one were to write down the exact domain to target space mapping (perhaps again in a truth table) of all possible balanced functions with a single atom for each mark in the case of $n=9$, one would be required access to not only every atom in our universe but also every atom in $10^{70}$ other universes of the same mass density. No further evidence should be required to demonstrate the futility of such a task.

Returning now to our quantum mechanical reality and the problem at hand, we can allow Bob to secretly (without Alice's awareness) select just a small subset of the possible balanced functions for any $n$. In addition to the two constant maps, Bob can choose only two of the possible balanced functions. With careful consideration, his choices for the two balanced functions would appropriately force any classical computer's solution attempt to make exponentially more queries to the oracle than Alice's appropriately constructed quantum algorithm. Bob's choices can additionally find an exact analogue to the inner workings of the oracle in the case of $n=1$ as we will later see. Bob's first appropriate balanced function choice follows as a mapping which takes the first $2^{n-1}$ bit strings to $0$, and the last $2^{n-1}$ bit strings to $1$. Likewise, his second choice for a balanced function is the target value swapping of the former. With these four choices for functions, a classical computer would be required to make $2^{n-1} + 1$ queries to the oracle in order to discern the the global property of the function.

tell the oracle the function's global property. As such, one removes the tedious process of considering all possible combinations for the available balanced and constant functions while constructing the inner workings of the blackbox in terms of quantum logic gates.

Indeed, one really need not know the full domain to target mapping of the function in order to devise a simplified version of this generalized Deutsch-Jozsa algorithm.

In [ ]:
# This generates a simplified function from {0,1}^n to {0,1} that is either constant or balanced according to the following logic:
# the function_type input is constant: we map the whole function to either 0 or 1 based on random number generation
# the function_type input is balanced: we map even indexed bit strings to zero and odd indexed bit strings to one

def bobs_function(n: int, function_type: str) -> List[int]:

  even_odd = random.randint(0,1)

  if function_type == "constant":
    return [even_odd] * (2**n)
  elif function_type == "balanced":
    # if we're mapping even indexes to 1 and odd indices to 0
    if even_odd == 1:
      return [int(i % 2 == 0) for i in range(2**n)]
    # vice versa
    else:
      return [int(i % 2 == 1) for i in range(2**n)]

In [83]:
# First, Bob builds a custom gate to encode the operation of his blackbox function to be used in the Alice's circuit
# The blackbox oracle, unbeknowst to the Alice, knows whether the function is constant or balanced based on Bob's choice
# **** #TODO
# Without telling Alice, Bob chooses one of the four restricted functions above

class bobs_deutsch_jozsa_oracle(cirq.Gate):

  def __init__(self, function_type: str, total_qubits: int):
        super(bobs_deutsch_jozsa_oracle, self)
        self.function_type = function_type
        self.total_system_qubits = total_qubits

  def _num_qubits_(self) -> int:
    return self.total_system_qubits

  def _decompose_(self, qubits) -> Iterable['cirq.Gate']:
    if self.function_type == "constant_0":
      for i in range(self.total_system_qubits - 1):
        yield cirq.IdentityGate(2).on(qubits[i], qubits[-1])
    elif self.function_type == "constant_1":
      for i in range(self.total_system_qubits - 1):
        yield cirq.X(qubits[i])
    elif self.function_type == "balanced_01":
      for i in range(self.total_system_qubits - 1):
        yield cirq.CNOT(qubits[i], qubits[-1])
    elif self.function_type == "balanced_10":
      for i in range(self.total_system_qubits - 1):
        # pass
        yield [cirq.CNOT(qubits[i], qubits[-1]), cirq.X(qubits[-1])]

  def _circuit_diagram_info_(self, args):
        return ["deutsch-jozsa"] * self._num_qubits_()

NameError: name 'Iterable' is not defined

In [60]:
# Now Alice designs her Deutsch-Jozsa quantum circuit
# She builds it to interact with the black box oracle that Bob will configure based on his choice for the global property of the function
# Her cirquit is a simple generalization of the early case for n = 1. Her system now has n qubits in a uniform superposition in her query register
# The oracle acts on all of them, applying a phase flip to a qubit if **** #TODO

def alices_deutsch_jozsa_algorithm(function_type: str, n: int, repetitions: int = 1) -> int:

  query_register = cirq.LineQubit.range(n)
  answer_qubit = cirq.LineQubit(n)

  # flip the answer qubit to |1>
  moment_0 = cirq.Moment(cirq.X(answer_qubit))

  # apply Hadamard gate to every qubit in system
  moment_1 = cirq.Moment(cirq.H.on_each([query_register, answer_qubit]))

  # apply the quantum deutsch-jozsa oracle to the full system register
  moment_2 = [cirq.Moment(bobs_deutsch_jozsa_oracle(function_type, n+1).on(*query_register, answer_qubit))]

  # apply Hadamard again to every qubit in the query register
  moment_3 = cirq.Moment(cirq.H.on_each(query_register))

  # measure the query registrer
  moment_4 = cirq.Moment(cirq.measure(query_register, key='result'))

  circuit = cirq.Circuit((moment_0, moment_1, moment_2, moment_3, moment_4))

  print(circuit)
  sim = cirq.Simulator()
  return sim.run(circuit, repetitions=repetitions)

In [82]:
# create the list of possible black box functions for the oracle
possible_dj_functions = ["constant_0", "constant_1", "balanced_01", "balanced_10"]

def format_output(final_states: list[int]) -> str:
  output = ""
  for i in final_states:
    output += "|" + str(i) + ">"
  return output

# Alice's dictionary that intelligently decodes her quantum algorithm's results
def dj_decoding(final_states: list[int]) -> str:
  zero_state = all(state == 0 for state in final_states)
  if zero_state:
    return "constant"
  else:
    return "balanced"

# Bob picks one of the possible functions at random
bobs_dj_choice = random.choice(possible_dj_functions)
print("Bob has randomly chosen a " + bobs_dj_choice + " function.\n")

# Alice and Bob agree on the number n
n = 6

# Alice runs her algorithm with a single query to the oracle to discern the global nature of the function
print("Making intelligent use of Bob's black box oracle, Alice constructs her quantum ciruit as so: \n")
dj_response = alices_deutsch_jozsa_algorithm(bobs_dj_choice, n)

print(f"\nShe runs her quantum algorithm and records her final {n} qubit query register in the following state: {format_output(dj_response.measurements['result'][0])}.\n")
print(f"She determines immediately, with just one query to the oracle, that the function is {dj_decoding(dj_response.measurements['result'][0]).upper()}.\n")

Bob has randomly chosen a balanced_even function.

Making intelligent use of Bob's black box oracle, Alice constructs her quantum ciruit as so: 

0: ───────H───deutsch-jozsa───H───M('result')───
              │                   │
1: ───────H───deutsch-jozsa───H───M─────────────
              │                   │
2: ───────H───deutsch-jozsa───H───M─────────────
              │                   │
3: ───────H───deutsch-jozsa───H───M─────────────
              │                   │
4: ───────H───deutsch-jozsa───H───M─────────────
              │                   │
5: ───────H───deutsch-jozsa───H───M─────────────
              │
6: ───X───H───deutsch-jozsa─────────────────────

She runs her quantum algorithm and records her final 6 qubit query register in the following state: |1>|1>|1>|1>|1>|1>.

She determines immediately, with just one query to the oracle, that the function is BALANCED.



In [1]:
# Again we allow Alice to run her experiment #repetitions times to confirm that it is fully deterministic using a single query to the oracle

repetitions = 15
# repetitions = 500
bobs_second_dj_choice = random.choice(possible_dj_functions)
print("This time, Bob has randomly chosen a " + bobs_second_dj_choice + " function.\n")

print("Again, Alice's quantum ciruit is: \n")

dj_response_2 = alices_deutsch_jozsa_algorithm(bobs_second_dj_choice, n, repetitions=repetitions)
print(f"\nAlice runs her algorithm {repetitions} times and determines in each case that that the function is:")
for i in range(repetitions):
  print(dj_decoding(dj_response_2.measurements['result'][0]).upper())

NameError: name 'random' is not defined

## Analyzing Claude's code generation for the Deutsch-Jozsa algorithm

Anthropic's LLM Claude has the capability to both express mathematical equations and

In [96]:
import cirq
import numpy as np

def deutsch_jozsa(n, oracle):
    # Create a circuit with n+1 qubits
    qubits = [cirq.LineQubit(i) for i in range(n+1)]
    circuit = cirq.Circuit()

    # Initialize the last qubit to |1>
    circuit.append(cirq.X(qubits[-1]))

    # Apply Hadamard gates to all qubits
    circuit.append(cirq.H.on_each(qubits))

    # Apply the oracle
    circuit.append(oracle(qubits))

    # Apply Hadamard gates to the first n qubits
    circuit.append(cirq.H.on_each(qubits[:-1]))

    # Measure the first n qubits
    circuit.append(cirq.measure(*qubits[:-1], key='result'))

    return circuit

# Example oracle functions
def constant_oracle(qubits):
    # Always return 0 (identity operation)
    return []

def balanced_oracle(qubits):
    # CNOT with the last qubit as control and all others as targets
    return [cirq.CNOT(qubits[-1], qubit) for qubit in qubits[:-1]]

# Run the algorithm
def run_deutsch_jozsa(n, oracle):
    circuit = deutsch_jozsa(n, oracle)
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=1)
    measurements = result.measurements['result'][0]
    is_constant = all(bit == 0 for bit in measurements)
    return "Constant" if is_constant else "Balanced"

# Test the algorithm
n = 3  # number of input qubits

print("Testing with constant oracle:")
print(run_deutsch_jozsa(n, constant_oracle))

print("\nTesting with balanced oracle:")
print(run_deutsch_jozsa(n, balanced_oracle))

Testing with constant oracle:
Constant

Testing with balanced oracle:
Constant
